# 물동량과 체류 시간의 상관관계 분석

### 데이터 로드, 전처리

In [8]:
import pandas as pd
import numpy as np
# 체류 시간(stayed_time_df_raw) 불러오기
stayed_time_df_raw=pd.read_csv('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data/shipSchedule(12 ~ 23).csv')

# 접안시간 기준으로 년도 행과 월 행 생성
year=[]
month=[]
for i in range(len(stayed_time_df_raw)):
    year.append(int(stayed_time_df_raw['접안시간'][i][:4]))
    month.append(int(stayed_time_df_raw['접안시간'][i][5:7]))
stayed_time_df_raw['년도']=year
stayed_time_df_raw['월']=month

# 년도와 월 행을 기준으로 groupby 후 체류 시간의 합 구하기
stayed_time_df=stayed_time_df_raw.groupby(['년도','월'],as_index=False)['체류시간_totalTime(시간)'].sum()

# 13년도 ~ 23년도 데이터만 추출
# 물동량 데이터가 13 ~ 23년도만 존재하기 때문
stayed_time_df=stayed_time_df[(stayed_time_df.년도>=2013) & (stayed_time_df.년도<=2023)].reset_index().drop('index',axis=1)

# 물동량 데이터 불러오기
shipment_df=pd.read_csv('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data/년,월별 물동량.csv')
shipment_df.drop(['Unnamed: 0','년/월'],axis=1,inplace=True)  #불필요한 행 제거

# 체류시간, 물동량 Merge
shipment_stayed_df=shipment_df.merge(stayed_time_df,on=['년도','월'])

# Merge한 데이터 저장
shipment_stayed_df.to_csv('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data/물동량-체류시간_상관분석.csv')

# 년도, 월 기준으로 sort
shipment_stayed_df.sort_values(['년도','월'],inplace=True)

### 물동량 - 체류시간 상관분석

In [9]:
# 상관계수_1
corr1=np.corrcoef(shipment_stayed_df['체류시간_totalTime(시간)'],shipment_stayed_df['컨테이너'])
corr1[1,0]

0.4933140536209876

### 시각화 - Plotly

##### 물동량 체류시간 상관분석 시각화 - Scatter

In [10]:
# 한글깨짐 방지
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False
#f_path = "/Library/Fonts/AppleGothic.ttf"
f_path = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)

In [11]:
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(x=shipment_stayed_df['체류시간_totalTime(시간)'],y=shipment_stayed_df['컨테이너'],
                         mode='markers',name='실측값',
                         # 호버 시 출력되는 정보 설정
                         hovertemplate='<b>년도:</b> %{text}',
                         # 호버탬플릿의 text 선언
                         text=shipment_stayed_df[['년도', '월']].apply(lambda x: f'{x[0]}년 {x[1]}월', axis=1)))

# 추세선을 추가합니다.
fig.add_trace(go.Scatter(
    x=shipment_stayed_df['체류시간_totalTime(시간)'],
    # 추세선 값
    y=np.poly1d(np.polyfit(shipment_stayed_df['체류시간_totalTime(시간)'], shipment_stayed_df['컨테이너'], 1))\
        (shipment_stayed_df['체류시간_totalTime(시간)']),
    mode='lines',line=dict(color='red', width=2),
    name='추세선'
))

fig.update_layout(title='물동량 - 체류시간',
                  xaxis=dict(title='체류시간(단위:시간)'),
                  yaxis=dict(title='물동량(단위:ton)'),
                  width=600,height=600,
                  # 마우스 위치의 x좌표를 기준으로 동일 위치에 있는 포인트의 정보 출력
                  hovermode='x',
                  # 상관계수 표시하기 위함
                  annotations=[
                      dict(x=-0.1,
                           y=1.1,
                           xref='paper',
                           yref='paper',
                           text=f'상관계수: {round(corr1[1,0],3)}',
                           showarrow=False)])

C:\Users\kamci\AppData\Local\Temp\ipykernel_11364\3587488162.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



##### +α 정규화 진행 후 시각화
* 정규화 하는 이유 : 단위에 따른 분석 결과의 오류를 없애기 위해

In [12]:
# 년/월 column 생성
# 정규화를 하기 위해선 column에 정규화를 진행할 변수만 남아 있어야 하므로
# 기존 데이터프레임에서 년/월 column을 생성 후 정규화를 진행한 데이터 프레임에 덧붙이기 위함
y_m=[]
for i in range(len(shipment_stayed_df)):
    y_m.append(str(shipment_stayed_df['년도'][i])+'-'+str(shipment_stayed_df['월'][i]))
shipment_stayed_df['년/월']=y_m
shipment_stayed_df['년/월']=pd.to_datetime(shipment_stayed_df['년/월'])
shipment_stayed_df=shipment_stayed_df.iloc[:,2:-1]

In [14]:
# minmaxscaler : 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()  #sclaer 선언
scaled_df=pd.DataFrame(
    scaler.fit_transform(shipment_stayed_df),columns=shipment_stayed_df.columns)  #체류시간, 물동량 정규화

In [24]:
scaled_df['년/월']=y_m

In [25]:
scaled_df

,컨테이너,체류시간_totalTime(시간),년/월
0,0.097572,0.010566,2013-1
1,0.000000,0.008595,2013-2
2,0.205185,0.015329,2013-3
3,0.197289,0.016862,2013-4
4,0.173105,0.015603,2013-5
...,...,...,...
127,0.598511,0.901785,2023-8
128,0.642204,0.881857,2023-9
129,0.578086,0.952480,2023-10
130,0.645874,0.858535,2023-11


##### +α_1 정규화 후 상관분석 및 산점도 시각화

In [26]:
# 상관계수_2
# pvalue : 상관계수가 유의미한 정도
# pvalue가 매우 작으므로 상관계수가 유의미하다고 할 수 있음
import scipy.stats as stats
corr2=stats.pearsonr(scaled_df['체류시간_totalTime(시간)'],scaled_df['컨테이너'])
corr2

PearsonRResult(statistic=0.4933140536209876, pvalue=1.8585102279594906e-09)

In [28]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=scaled_df['체류시간_totalTime(시간)'],y=scaled_df['컨테이너'],
                         hovertemplate='<b>년도:</b> %{text}',
                         mode='markers',name='실측값',
                         text=scaled_df['년/월']))

# 추세선을 추가합니다.
fig.add_trace(go.Scatter(
    x=scaled_df['체류시간_totalTime(시간)'],
    y=np.poly1d(np.polyfit(scaled_df['체류시간_totalTime(시간)'], scaled_df['컨테이너'],1))\
        (scaled_df['체류시간_totalTime(시간)']),
    mode='lines',line=dict(color='red', width=2),
    name='추세선'
))

fig.update_layout(title='물동량 - 체류시간(정규화)',
                  xaxis=dict(title='체류시간'),
                  yaxis=dict(title='물동량'),
                  width=600,height=600,
                  hovermode='x',
                  annotations=[
                      dict(x=-0.1,
                           y=1.1,
                           xref='paper',
                           yref='paper',
                           text=f'상관계수: {round(corr2[0],3)}',
                           showarrow=False)])

* 상관계수 값과 산점도가 동일한 것으로 보아 상관계수 계산 모듈이 정규화를 진행하는 것으로 보임

##### +α_2 정규화 후 시간에 흐름에 따른 물동량과, 체류시간의 증감 추세 파악

In [29]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=scaled_df['년/월'],y=scaled_df['컨테이너'],name='물동량'))
fig.add_trace(go.Scatter(x=scaled_df['년/월'],y=scaled_df['체류시간_totalTime(시간)'],name='체류시간'))
fig.update_layout(title='물동량 / 체류시간',
                  xaxis=dict(title='년/월'))

# 결론

#### 1. 체류 시간과 물동량 간의 상관 계수는 0.493으로 양의 상관관계를 가진다
#### 2. 시간이 지남에 따라 체류 시간이 꾸준히 증가 추세를 보이고 있다
#### 3. 물동량은 코로나 발발 시점 이후 크게 감소했으나, 상관계수를 고려했을 때 물동량도 증가할 것으로 예상할 수 있다
#### 4. 외국인 선원들의 체류시간의 증가 추세를 예상할 수 있으므로 부산항만공사에게 외국인 선윈 대상 마케팅 전략 인사이트를 제공하고자 함